When doing prepare dataset following specs are required
* 64 GB Ram
* 300 GB Disk Storage

When doing training the recomended specs are:
* 24GB GPU RAM
* 1 GPU (Nvidia GTX 4090 or better)
* 300 GB of free disk space

In [ ]:
#https://towardsdatascience.com/leveraging-the-power-of-jupyter-notebooks-26b4b8d7c622
 jupyter notebook --generate-config
 jupyter notebook --NotebookApp.max_buffer_size=258000000000
 jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

In [1]:
%pip install pandas 
%pip install datasets
%pip install transformers
%pip install librosa
%pip install wandb -qU
%pip install git+https://github.com/huggingface/huggingface_hub
%pip install jiwer
%pip install transformers[torch]
%pip install accelerate -U
%pip install ipywidgets
%pip install torchaudio

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.3/417.3 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 35.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.0
    Uninstalling fsspec-2024.6.0:
      Successfully uninstalled fsspec-2024.6.0
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 46.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 111.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use u

In [2]:
!pip install wandb --upgrade

In [3]:
!rm wandb --recursive 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc


True

In [ ]:
import os

# Set the WANDB_NOTEBOOK_NAME environment variable
os.environ["WANDB_NOTEBOOK_NAME"] = "train-wav2vec2.ipynb"

In [ ]:
wandb.login(relogin=True)

In [ ]:
wandb.init(project="wav2vec2-training", entity="TMLM", name="mms_300_v2")

In [5]:
import pandas as pd
import numpy as np
import random
from IPython.display import display, HTML
import os
import json
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
import IPython.display as ipd
import torchaudio
from transformers import AutoProcessor

In [6]:
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor
from datasets import load_dataset, load_metric
from datasets import Dataset
from datasets import ClassLabel
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import TrainingArguments
from transformers import Trainer
from jiwer import wer
import statistics
from transformers import Wav2Vec2CTCTokenizer

In [ ]:
!aws s3 ls monlam.ai.stt/tsv/

In [ ]:
! mkdir tsv
! curl https://d38pmlk0v88drf.cloudfront.net/tsv/06_training.csv --output tsv/training.csv
! curl https://d38pmlk0v88drf.cloudfront.net/tsv/06_validation.csv --output tsv/validation.csv
! curl https://d38pmlk0v88drf.cloudfront.net/tsv/05_benchmarkings.csv --output tsv/test.csv


In [ ]:
dataTrain = pd.read_csv("tsv/training.csv")
dataValid = pd.read_csv("tsv/validation.csv")
dataTest = pd.read_csv("tsv/test.csv")

In [ ]:
len(dataTrain), len(dataValid), len(dataTest)

In [ ]:
! aws s3 sync s3://monlam.ai.stt/wav16k wav16k

In [ ]:
pd.options.mode.chained_assignment = None
dataTest['path'] = dataTest['file_name'].apply(lambda x: f'/home/ec2-user/Sagemaker/stt-wav2vec2/wav16k/{x}.wav')

dataValid['path'] = dataValid['file_name'].apply(lambda x: f'/home/ec2-user/Sagemaker/stt-wav2vec2/wav16k/{x}.wav')

dataTrain['path'] = dataTrain['file_name'].apply(lambda x: f'/home/ec2-user/Sagemaker/stt-wav2vec2/wav16k/{x}.wav')

In [ ]:
import os
dataTest['path'].apply(lambda x: os.path.isfile(x)).value_counts()
# dataValid['path'].apply(lambda x: os.path.isfile(x)).value_counts()
# batch_df['path'].apply(lambda x: os.path.isfile(x)).value_counts()

In [ ]:
from datasets import Dataset
common_voice_train = Dataset.from_pandas(dataTrain)
common_voice_valid = Dataset.from_pandas(dataValid)
common_voice_test = Dataset.from_pandas(dataTest)

common_voice_test_transcription = Dataset.from_pandas(dataTest)
common_voice_valid_transcription = Dataset.from_pandas(dataValid)

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(['dept', 'grade', 'wylie', 'char_len', 'audio_len', 'url']), num_examples=5)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["uni"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_valid = common_voice_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_valid.column_names)

In [ ]:
vocab_list = list(
    set(vocab_train["vocab"][0]) | 
    set(vocab_test ["vocab"][0]) | 
    set(vocab_valid["vocab"][0])
    )

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
# ! aws s3 cp vocab.json s3://monlam.ai.stt/tsv/vocab.json

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("mms_300_v2")

In [ ]:
from torchaudio.transforms import Resample

def speech_file_to_array_fn(batch):
    # print(batch)
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    # print(speech_array.shape, sampling_rate)
    if sampling_rate != 16000:
        print("resampling")
        resampler = Resample(orig_freq=sampling_rate, new_freq=16000)
        speech_array = resampler(speech_array)
        sampling_rate = 16000
    
    # print(speech_array.shape, sampling_rate)
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["uni"]
    return batch

In [ ]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)

In [ ]:
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

In [ ]:
common_voice_valid = common_voice_valid.map(speech_file_to_array_fn, remove_columns=common_voice_valid.column_names)

humm. This does not work. 

In [ ]:
common_voice_train

In [ ]:
rand_int = random.randint(0, len(common_voice_test)-1)

ipd.Audio(data=np.asarray(common_voice_test[rand_int]["path"]), autoplay=True, rate=16000)

In [ ]:
# rand_int = random.randint(0, len(common_voice_test)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"]).input_values
    # reshape to (n,)
    batch["input_values"] = np.squeeze(batch["input_values"])
    # if batch["sampling_rate"] != 16000:
    #     print("sampling rate not 16k", batch)
    
    # with processor.as_target_processor():
    #     batch["labels"] = processor(batch["target_text"]).input_ids

    batch["labels"] = processor(text=batch["target_text"]).input_ids
    return batch

In [ ]:
# def prepare_dataset(batch):
#     # check that all files have the correct sampling rate
#     assert (
#         len(set(batch["sampling_rate"])) == 1
#     ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

#     batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

#     with processor.as_target_processor():
#         batch["labels"] = processor(batch["target_text"]).input_ids
#     return batch

# common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=4, num_proc=2, batched=True)
# common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)
# common_voice_valid = common_voice_valid.map(prepare_dataset, remove_columns=common_voice_valid.column_names, batch_size=8, num_proc=4, batched=True)

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_train.save_to_disk(f"/media/monlamai/SSD/wav2vec2/train_prepare_dataset.arrow")

In [ ]:
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)
common_voice_test.save_to_disk("/media/monlamai/SSD/wav2vec2/test_prepare_dataset.arrow")

In [ ]:
common_voice_valid = common_voice_valid.map(prepare_dataset, remove_columns=common_voice_valid.column_names)
common_voice_valid.save_to_disk("/media/monlamai/SSD/wav2vec2/valid_prepare_dataset.arrow")

In [ ]:
# from datasets import DatasetDict
# ddict = DatasetDict({
#     "train": common_voice_train,
#     "valid": common_voice_valid,
#     "test": common_voice_test,
# })
# ddict.push_to_hub("prepare_dataset_run8")

### Load the datasets from disk

In [ ]:
!rm -r wav16k 

In [ ]:
! aws s3 cp s3://monlam.ai.stt/wav16k/STT_MV0833_0298_2228254_to_2237817.wav STT_MV0833_0298_2228254_to_2237817.wav
! aws s3 cp s3://monlam.ai.stt/wav16k/STT_MV0833_0291_2194075_to_2195733.wav STT_MV0833_0291_2194075_to_2195733.wav
! aws s3 cp s3://monlam.ai.stt/wav16k/STT_MV0833_0293_2205438_to_2212722.wav STT_MV0833_0293_2205438_to_2212722.wav

In [ ]:
! aws s3 ls s3://monlam.ai.stt/dataset/wav2vec2/train_prepare_dataset.arrow/

In [7]:
from datasets import load_from_disk
common_voice_train = load_from_disk('/home/ec2-user/SageMaker/stt-wav2vec2/data/train_prepare_dataset.arrow')
common_voice_test =  load_from_disk( '/home/ec2-user/SageMaker/stt-wav2vec2/data/test_prepare_dataset.arrow')
common_voice_valid = load_from_disk('/home/ec2-user/SageMaker/stt-wav2vec2/data/valid_prepare_dataset.arrow')

Loading dataset from disk:   0%|          | 0/1071 [00:00<?, ?it/s]

In [8]:
common_voice_train[0]

{'file_name': 'STT_TT00434_58.900-60.650',
 'uni': 'དེ་འདྲའི་གི་ཐོག་ནས་ང་ལས་དམའ་དམའ་གི་ ',
 'wylie': "de 'dra'i gi thog nas nga las dma' dma' gi _",
 'url': 'https://d38pmlk0v88drf.cloudfront.net/wav16k/STT_TT00434_58.900-60.650.wav',
 'dept': 'STT_TT',
 'grade': 3,
 'char_len': 36,
 'audio_len': 1.75,
 'path': 'wav16k/STT_TT00434_58.900-60.650.wav',
 '__index_level_0__': 0,
 'speech': [0.0135498046875,
  0.008087158203125,
  0.000732421875,
  -0.00506591796875,
  -0.0098876953125,
  -0.015655517578125,
  -0.019744873046875,
  -0.02008056640625,
  -0.018096923828125,
  -0.01513671875,
  -0.01104736328125,
  -0.005645751953125,
  0.000396728515625,
  0.006744384765625,
  0.013580322265625,
  0.019805908203125,
  0.02435302734375,
  0.02734375,
  0.027008056640625,
  0.0218505859375,
  0.014190673828125,
  0.006561279296875,
  0.00042724609375,
  -0.00360107421875,
  -0.007720947265625,
  -0.012664794921875,
  -0.0155029296875,
  -0.015167236328125,
  -0.01373291015625,
  -0.013702392578

In [9]:
common_voice_train[0]['labels']

[15,
 23,
 112,
 11,
 15,
 105,
 11,
 18,
 112,
 116,
 18,
 112,
 59,
 58,
 116,
 112,
 24,
 71,
 112,
 20,
 112,
 73,
 71,
 112,
 15,
 80,
 11,
 112,
 15,
 80,
 11,
 112,
 116,
 18,
 112,
 70]

In [10]:
from transformers import AutoProcessor

In [12]:
processor = AutoProcessor.from_pretrained('mms_300_v2')

In [13]:
ls 

batch_data/      mms_300_v2/                  train-wav2vec2.ipynb
CONTRIBUTING.md  prepare_dataset-Copy2.ipynb  tsv/
data/            prepare_dataset.ipynb        vocab.json
error_log.log    README.md                    wav16k/
LICENSE.md       result/
mms_300/         train-wav2vec2-Copy1.ipynb


In [14]:
processor.tokenizer.decode(common_voice_train[89]['labels'])

'བསམ་བློ་ག་རེ་གཏང་གི་ཡོད་ ང་རང་ཚོའི་ལག་རྩལ་འདི་བྱས་ལམ་ཁ་སྡུག་ག་གཅིག་ཡོད་རེད་ རི་ཁུལ་ལ་དཔར་བརྒྱབ་ཡོང་ང། ད་དེ་ལས་རྙོག་དྲ་མ་སློང་ ཨ་ཙི་'

In [15]:
common_voice_train[0]['input_values']

[0.05832287296652794,
 0.03490390628576279,
 0.0033733448944985867,
 -0.021484773606061935,
 -0.04215626418590546,
 -0.06688354909420013,
 -0.08441506326198578,
 -0.08585421741008759,
 -0.07735012471675873,
 -0.06465940177440643,
 -0.04712788760662079,
 -0.0239705853164196,
 0.00193419074639678,
 0.02914728969335556,
 0.05845370516180992,
 0.08514347672462463,
 0.10463747382164001,
 0.11745902895927429,
 0.11601987481117249,
 0.09390923380851746,
 0.06107034906744957,
 0.028362296521663666,
 0.0020650229416787624,
 -0.01520482823252678,
 -0.032867174595594406,
 -0.05406199395656586,
 -0.06622938811779022,
 -0.06479023396968842,
 -0.05864112079143524,
 -0.05851028859615326,
 -0.06361274421215057,
 -0.05641697347164154,
 -0.030773861333727837,
 -0.00591574190184474,
 0.011354110203683376,
 0.024437330663204193,
 0.028231464326381683,
 0.029539786279201508,
 0.03568889945745468,
 0.0355580672621727,
 0.03124060481786728,
 0.035296402871608734,
 0.04380049556493759,
 0.05335124954581261,
 

In [16]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [17]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [18]:
from datasets import load_metric
cer_metric = load_metric("cer")

/tmp/ipykernel_5876/1679503157.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


The repository for cer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [19]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [20]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    # "/media/monlamai/SSD/mms_300/mms_300_v1/checkpoint-1140000", # inserted for continue training
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    # gradient_checkpointing=True, # If True, use gradient checkpointing to save memory at the expense of slower backward pass.
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id, # commented for for continue training
    vocab_size=len(processor.tokenizer), # commented for for continue training
    # ignore_mismatched_sizes=True,
)

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model.freeze_feature_encoder()

In [22]:
model.config.ctc_zero_infinity = True

In [23]:
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir="./result",
  group_by_length=True,
  per_device_train_batch_size=8, # 
  gradient_accumulation_steps=2, # increase by 2x for every 2x decrease in batch size
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=10000,
  eval_steps=10000,
  logging_steps=20,
  report_to=['wandb'],
  learning_rate=3e-5,
  warmup_steps=50,
  save_total_limit=40,
  push_to_hub=False,
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_valid,
    tokenizer=processor.feature_extractor,
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# trainer.train(resume_from_checkpoint=True)  # uncomment for continue training
trainer.train()

In [ ]:
print("t")

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("openpecha/wav2vec2_run10").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("openpecha/wav2vec2_run10")

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())


In [ ]:
prediction = []
reference = []
paths = []

for i in range(0,len(common_voice_test)):

  input_dict = processor(common_voice_test[i]["input_values"], return_tensors="pt", padding=True)
  logits = model(input_dict.input_values.to("cuda")).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]

  #print("Prediction:")
  prediction.append(processor.decode(pred_ids))

  #print("\nReference:")
  reference.append(common_voice_test_transcription[i]["sentence"].lower())

  path = common_voice_test_transcription[i]["path"]
  path = path.split("/")
  path = path[-1]
  paths.append(path)

In [ ]:
for i in range(0,len(reference)):
  print(paths[i])
  print(reference[i])
  print(prediction[i])
  print("---")

In [ ]:
# This are necessary for the statistics reporting
from google.colab import files
import re
from jiwer import wer
import statistics

In [ ]:
# Calculate Levenshtein Distance between two strings (character distance)
# https://colab.research.google.com/github/Alexjmsherman/nlp_practicum_cohort3_instructor/blob/master/lessons/lesson_8_text_similarity/text_similarity_solution.ipynb#scrollTo=sSj3zYpq-sc1

def levenshtein(seq1, seq2):
    # create a matrix
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))

    # set col numbers (0, n-1)
    for x in range(size_x):
        matrix [x, 0] = x

    # set row numbers (0, n-1)
    for y in range(size_y):
        matrix [0, y] = y

    # calculate distance
    for x in range(1, size_x):
        for y in range(1, size_y):
            # if characters match do not increase distance
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = matrix[x-1, y-1]
            # if characters don't match increase min distance by 1
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )

    return (matrix[size_x - 1, size_y - 1])

In [ ]:
#===============================================================================
# Evaluate checkpoints; calculate their word/character error rates and
# get the predictions for the sentences in the test set.
#===============================================================================

checkpointNums = ["2900"]

medianStats = ""

for ch in checkpointNums:

	checkpointNum = ch

	filename = "wav2vec2-res-" + str(runId) + "-ch" + ch + ".csv"
	idThisRun = "wav2vec2-" + str(runId)

	# model = Wav2Vec2ForCTC.from_pretrained("/content/wav2vec2-large-xlsr/checkpoint-"+ch).to("cuda")
	# processor = Wav2Vec2Processor.from_pretrained("/content/wav2vec2-large-xlsr")

	input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)
	logits = model(input_dict.input_values.to("cuda")).logits
	pred_ids = torch.argmax(logits, dim=-1)[0]

	prediction = []
	reference = []
	paths = []

	for i in range(0,len(common_voice_test)):

		input_dict = processor(common_voice_test[i]["input_values"], return_tensors="pt", padding=True)
		logits = model(input_dict.input_values.to("cuda")).logits
		pred_ids = torch.argmax(logits, dim=-1)[0]

		#print("Prediction:")
		prediction.append(processor.decode(pred_ids))

		#print("\nReference:")
		reference.append(common_voice_test_transcription[i]["sentence"].lower())

		path = common_voice_test_transcription[i]["path"]
		path = path.split("/")
		path = path[-1]
		paths.append(path)

	output = "wav,src,res,loss,charDist,charLen,wordDist,wordLen,cer,wer,origin,condition,id,typeMonoTri,ngram\n"
	cerList = []
	werList = []

	for i in range(0,len(reference)):

		levDistChar = levenshtein(reference[i],prediction[i])
		cer = levDistChar / len(reference[i])

		werSent = wer(reference[i],prediction[i])
		charLen = len(reference[i])
		charDist = levDistChar
		wordLen = len(prediction[i].split(' '))
		wordDist = werSent*wordLen

		cerList.append(cer)
		werList.append(werSent)

		wavFile = paths[i].replace(".wav","")

		output += wavFile + "," + reference[i] + "," + prediction[i] + ",," + str(charDist) + "," + str(charLen) + "," + str(wordDist) + "," + str(wordLen) + "," + str(round(cer,2)) + "," + str(round(werSent,2)) + "," + "wav2vec2" + "," + "standard-" + ch + "," + str(idThisRun) + "," + "na" + "," + "na" + "\n"

	output = output[:-1]
	#print(output)

	cerMedian = statistics.median(cerList)
	werMedian = statistics.median(werList)

	medianStats += runId + "/" + ch + " Median CER:\t" + str(round(cerMedian,3)) + "\n"
	medianStats += runId + "/" + ch + " Median WER:\t" + str(round(werMedian,3)) + "\n\n"

	#print(runId + "/" + ch + " Median CER:\t" + str(round(cerMedian,3)))
	#print(runId + "/" + ch + " Median WER:\t" + str(round(werMedian,3)))

	print(output)

print(medianStats)

In [ ]:
medianStats = "Run: " + runId + "\n\n" + medianStats

statsFilename = "wav2vec2-res-"+str(runId)+"-stats-median.txt"
f = open(datasetPath + "logs-wav2vec2-res/" + statsFilename, "w")
f.write(medianStats)
f.close()

print(medianStats)

In [ ]:
# Visualization of CER and WER
df = pd.read_csv(datasetPath + "logs-wav2vec2-res/" + filename)
df.boxplot(by =['origin'], column =['cer','wer'], grid = False)

# For quick visualization only (display only sentences with CER and WER less than 2)
dfOnlyLessThanTwo = df[df['cer']<2]
dfOnlyLessThanTwo = dfOnlyLessThanTwo[dfOnlyLessThanTwo['wer']<2]
dfOnlyLessThanTwo.boxplot(by =['origin'], column =['cer','wer'], grid = False)